In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn import metrics

Lendo dados

In [26]:
data = pd.read_csv('data/brasil_2022.csv', sep=';', encoding='latin1')
data_copy = data.copy()
# df_limpo = data.loc[df['x'] != ""]

In [28]:
data_copy

,Região/Unidade da Federação,Menor de 10 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,Idade ignorada
0,Região Norte,1,46,581.0,808.0,696.0,515.0,391.0,156.0,16,1,-,128.0
1,.. Rondônia,-,3,33.0,54.0,56.0,26.0,28.0,13.0,1,-,-,8.0
2,.. Acre,-,3,24.0,33.0,28.0,22.0,16.0,11.0,1,-,-,14.0
3,.. Amazonas,1,12,145.0,199.0,167.0,141.0,110.0,54.0,2,-,-,21.0
4,.. Roraima,-,-,27.0,18.0,29.0,26.0,14.0,3.0,-,-,-,5.0
5,.. Pará,-,22,269.0,421.0,352.0,239.0,168.0,55.0,7,1,-,62.0
6,.. Amapá,-,1,33.0,28.0,29.0,25.0,21.0,13.0,5,-,-,9.0
7,.. Tocantins,-,5,50.0,55.0,35.0,36.0,34.0,7.0,-,-,-,9.0
8,Região Nordeste,-,96,1125.0,1802.0,1748.0,1597.0,1242.0,499.0,41,3,1,500.0
9,.. Maranhão,-,20,207.0,306.0,273.0,231.0,152.0,48.0,6,1,-,71.0


In [36]:
data_copy['Região'] = ''

# Armazena o último valor de grupo principal encontrado
ultimo_valor_principal = ''

for index, row in data_copy.iterrows():
    if '..' not in row['Região/Unidade da Federação']:
        # Atualiza o último valor principal se a linha atual não é um subgrupo
        ultimo_valor_principal = row['Região/Unidade da Federação']
    # Atualiza a nova coluna com o valor do último grupo principal encontrado
    data_copy.at[index, 'Região'] = ultimo_valor_principal


data_copy.head()


,Região/Unidade da Federação,Menor de 10 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,Idade ignorada,Região
0,Região Norte,1,46,581.0,808.0,696.0,515.0,391.0,156.0,16,1,-,128.0,Região Norte
1,.. Rondônia,-,3,33.0,54.0,56.0,26.0,28.0,13.0,1,-,-,8.0,Região Norte
2,.. Acre,-,3,24.0,33.0,28.0,22.0,16.0,11.0,1,-,-,14.0,Região Norte
3,.. Amazonas,1,12,145.0,199.0,167.0,141.0,110.0,54.0,2,-,-,21.0,Região Norte
4,.. Roraima,-,-,27.0,18.0,29.0,26.0,14.0,3.0,-,-,-,5.0,Região Norte


In [46]:
regions = data_copy['Região'].unique()
filtered_data = data_copy[~data_copy['Região/Unidade da Federação'].isin(regions)]

filtered_data['Região/Unidade da Federação'] = filtered_data['Região/Unidade da Federação'].str.replace('^\.\.\ ', '', regex=True)
filtered_data
filtered_data.head()

/tmp/ipykernel_9083/802098182.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Região/Unidade da Federação'] = filtered_data['Região/Unidade da Federação'].str.replace('^\.\.\ ', '', regex=True)


,Região/Unidade da Federação,Menor de 10 anos,10 a 14 anos,15 a 19 anos,20 a 24 anos,25 a 29 anos,30 a 34 anos,35 a 39 anos,40 a 44 anos,45 a 49 anos,50 a 54 anos,55 a 59 anos,Idade ignorada,Região
1,Rondônia,-,3,33.0,54.0,56.0,26.0,28.0,13.0,1,-,-,8.0,Região Norte
2,Acre,-,3,24.0,33.0,28.0,22.0,16.0,11.0,1,-,-,14.0,Região Norte
3,Amazonas,1,12,145.0,199.0,167.0,141.0,110.0,54.0,2,-,-,21.0,Região Norte
4,Roraima,-,-,27.0,18.0,29.0,26.0,14.0,3.0,-,-,-,5.0,Região Norte
5,Pará,-,22,269.0,421.0,352.0,239.0,168.0,55.0,7,1,-,62.0,Região Norte


In [47]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 1 to 31
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Região/Unidade da Federação  27 non-null     object 
 1   Menor de 10 anos             27 non-null     object 
 2   10 a 14 anos                 27 non-null     object 
 3   15 a 19 anos                 27 non-null     float64
 4   20 a 24 anos                 27 non-null     float64
 5   25 a 29 anos                 27 non-null     float64
 6   30 a 34 anos                 27 non-null     float64
 7   35 a 39 anos                 27 non-null     float64
 8   40 a 44 anos                 27 non-null     float64
 9   45 a 49 anos                 27 non-null     object 
 10  50 a 54 anos                 27 non-null     object 
 11  55 a 59 anos                 27 non-null     object 
 12  Idade ignorada               27 non-null     float64
 13  Região                     

In [52]:
filtered_data = filtered_data.replace('-', np.nan)
filtered_data.fillna(0, inplace=True)
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 1 to 31
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Região/Unidade da Federação  27 non-null     object 
 1   Menor de 10 anos             27 non-null     object 
 2   10 a 14 anos                 27 non-null     object 
 3   15 a 19 anos                 27 non-null     float64
 4   20 a 24 anos                 27 non-null     float64
 5   25 a 29 anos                 27 non-null     float64
 6   30 a 34 anos                 27 non-null     float64
 7   35 a 39 anos                 27 non-null     float64
 8   40 a 44 anos                 27 non-null     float64
 9   45 a 49 anos                 27 non-null     object 
 10  50 a 54 anos                 27 non-null     object 
 11  55 a 59 anos                 27 non-null     object 
 12  Idade ignorada               27 non-null     float64
 13  Região                     

In [66]:
filtered_data.iloc[:, 1:13] = filtered_data.iloc[:, 1:13].astype('int64')
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, 1 to 31
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Região/Unidade da Federação  27 non-null     object 
 1   Menor de 10 anos             27 non-null     object 
 2   10 a 14 anos                 27 non-null     object 
 3   15 a 19 anos                 27 non-null     float64
 4   20 a 24 anos                 27 non-null     float64
 5   25 a 29 anos                 27 non-null     float64
 6   30 a 34 anos                 27 non-null     float64
 7   35 a 39 anos                 27 non-null     float64
 8   40 a 44 anos                 27 non-null     float64
 9   45 a 49 anos                 27 non-null     object 
 10  50 a 54 anos                 27 non-null     object 
 11  55 a 59 anos                 27 non-null     object 
 12  Idade ignorada               27 non-null     float64
 13  Região                     

In [27]:
# data_copy = data_copy.loc[data_copy['Unidade da Federação'] != " "]
# data_copy

KeyError: 'Unidade da Federação'

In [25]:
df_limpo['Unidade da Federação'].value_counts()

Unidade da Federação
11 Rondônia               1
12 Acre                   1
13 Amazonas               1
14 Roraima                1
15 Pará                   1
16 Amapá                  1
17 Tocantins              1
21 Maranhão               1
22 Piauí                  1
23 Ceará                  1
24 Rio Grande do Norte    1
25 Paraíba                1
26 Pernambuco             1
27 Alagoas                1
28 Sergipe                1
29 Bahia                  1
31 Minas Gerais           1
32 Espírito Santo         1
33 Rio de Janeiro         1
35 São Paulo              1
41 Paraná                 1
42 Santa Catarina         1
43 Rio Grande do Sul      1
50 Mato Grosso do Sul     1
51 Mato Grosso            1
52 Goiás                  1
53 Distrito Federal       1
Name: count, dtype: int64